In [99]:
import numpy as np
import matplotlib.pyplot as py
import MontagePy as Montage
from MontagePy.main import *
from MontagePy.archive import *
from IPython.display import Image
from astroquery.mast import Catalogs
from astroquery.mast import Observations
import pandas as pd
from astropy.table import QTable
from astropy import units as u
from astropy.coordinates import SkyCoord
import os
import shutil

In [100]:
# Specify the intial path where you will be creating a new directory
path_init = '/home/akgh2880/research'
os.chdir(path_init)

In [101]:
dir_name = 'Bubble Nebula' # Give it a directory name; e.g. name of the target
os.makedirs(dir_name)

# Changes to the new directory just created
path = path_init + '/' + dir_name
os.chdir( path )  # Changes directory 
raw_dir = 'raw'
os.makedirs(raw_dir)
os.getcwd() # Shows the current working directory

'/home/akgh2880/research/Bubble Nebula'

In [102]:
path_raw = path + '/' + raw_dir
os.chdir(path_raw)  # Goes to the raw directory
os.getcwd()

'/home/akgh2880/research/Bubble Nebula/raw'

In [103]:
r = 0.5 # Search radius of query
# Input coordinates below 
RA = '23:20:48'
DEC = '+61:12:6'
c = SkyCoord(RA, DEC, unit=(u.hourangle, u.deg))
#c = SkyCoord('23:20:48', '+61:12:6', unit=(u.hourangle, u.deg))

# Runs GALEX query at the given coordinate 
result_table = Observations.query_criteria(coordinates=c, obs_collection='GALEX', radius=r)# filters=["*UV"])
obsid = result_table ['obsid']

In [104]:
print(obsid)

obsid
-----
10950
10951
11026
11027


In [105]:
Observations.get_product_list(obsid)

obsID,obs_collection,dataproduct_type,obs_id,description,type,dataURI,productType,productGroupDescription,productSubGroupDescription,productDocumentationURL,project,prvversion,proposal_id,productFilename,size,parent_obsid,dataRights,calib_level
str5,str5,str5,str19,str139,str1,str135,str9,str28,str12,str1,str3,str1,str1,str55,int64,str5,str6,int64
75066,GALEX,image,6371830505138028544,Catalog of matches between -flagstar and -[n/f]-cat files,S,mast:GALEX/url/data/GR6/pipe/02-vsn/50026-AIS_26/d/00-visits/0001-img/07-try/AIS_26_0001_sg87-nd-cat_mch_flagstar.fits.gz,AUXILIARY,--,--,--,AIS,--,--,AIS_26_0001_sg87-nd-cat_mch_flagstar.fits.gz,349515,10950,PUBLIC,2
75066,GALEX,image,6371830505138028544,Count map (J2000),S,mast:GALEX/url/data/GR6/pipe/02-vsn/50026-AIS_26/d/00-visits/0001-img/07-try/AIS_26_0001_sg87-nd-cnt.fits.gz,AUXILIARY,--,--,--,AIS,--,--,AIS_26_0001_sg87-nd-cnt.fits.gz,2807786,10950,PUBLIC,2
75066,GALEX,image,6371830505138028544,Exposure map (J2000),S,mast:GALEX/url/data/GR6/pipe/02-vsn/50026-AIS_26/d/00-visits/0001-img/07-try/AIS_26_0001_sg87-nd-exp.fits.gz,AUXILIARY,--,--,--,AIS,--,--,AIS_26_0001_sg87-nd-exp.fits.gz,63671,10950,PUBLIC,2
75066,GALEX,image,6371830505138028544,Artifact and source star detailed info.,S,mast:GALEX/url/data/GR6/pipe/02-vsn/50026-AIS_26/d/00-visits/0001-img/07-try/AIS_26_0001_sg87-nd-flag_tbl.fits.gz,AUXILIARY,--,--,--,AIS,--,--,AIS_26_0001_sg87-nd-flag_tbl.fits.gz,129143,10950,PUBLIC,2
75066,GALEX,image,6371830505138028544,Artifact Flag image (J2000),S,mast:GALEX/url/data/GR6/pipe/02-vsn/50026-AIS_26/d/00-visits/0001-img/07-try/AIS_26_0001_sg87-nd-flags.fits.gz,AUXILIARY,--,--,--,AIS,--,--,AIS_26_0001_sg87-nd-flags.fits.gz,56523,10950,PUBLIC,2
75066,GALEX,image,6371830505138028544,Star catalog for flagging,S,mast:GALEX/url/data/GR6/pipe/02-vsn/50026-AIS_26/d/00-visits/0001-img/07-try/AIS_26_0001_sg87-nd-flagstar.fits.gz,AUXILIARY,--,--,--,AIS,--,--,AIS_26_0001_sg87-nd-flagstar.fits.gz,155053,10950,PUBLIC,2
75066,GALEX,image,6371830505138028544,Background subtracted intensity map (J2000),S,mast:GALEX/url/data/GR6/pipe/02-vsn/50026-AIS_26/d/00-visits/0001-img/07-try/AIS_26_0001_sg87-nd-intbgsub.fits.gz,AUXILIARY,--,--,--,AIS,--,--,AIS_26_0001_sg87-nd-intbgsub.fits.gz,15826665,10950,PUBLIC,2
75066,GALEX,image,6371830505138028544,Time-slice count maps (J2000),S,mast:GALEX/url/data/GR6/pipe/02-vsn/50026-AIS_26/d/00-visits/0001-img/07-try/AIS_26_0001_sg87-nd-movie.fits.gz,AUXILIARY,--,--,--,AIS,--,--,AIS_26_0001_sg87-nd-movie.fits.gz,637668,10950,PUBLIC,2
75066,GALEX,image,6371830505138028544,Masked field image (J2000),S,mast:GALEX/url/data/GR6/pipe/02-vsn/50026-AIS_26/d/00-visits/0001-img/07-try/AIS_26_0001_sg87-nd-objmask.fits.gz,AUXILIARY,--,--,--,AIS,--,--,AIS_26_0001_sg87-nd-objmask.fits.gz,152401,10950,PUBLIC,2


In [106]:
data_products = Observations.get_product_list(obsid)

# Converts data products to panda data frame to execute suffix search
data_products =  data_products.to_pandas()

# Finds cnt, rrhr, and skybg files and stores as DP_cnt, DP_rrhr, and DP_skybg (= suffix search)
DP_cnt = data_products[(data_products['productFilename'].str.endswith('-cnt.fits.gz'))] 
DP_rrhr = data_products[(data_products['productFilename'].str.endswith('-rrhr.fits.gz'))] 
DP_skybg = data_products[(data_products['productFilename'].str.endswith('-skybg.fits.gz'))] 

# Convert data frame back to QTable
DP_cnt = QTable.from_pandas(DP_cnt)
DP_rrhr = QTable.from_pandas(DP_rrhr)
DP_skybg = QTable.from_pandas(DP_skybg)

In [107]:
# Download selected files
manifest = Observations.download_products(DP_cnt)
manifest = Observations.download_products(DP_rrhr)
manifest = Observations.download_products(DP_skybg)


In [108]:
extension_dir = 'mastDownload/GALEX' # Directories that get automatically created by MAST query
os.chdir(path + '/' + raw_dir + '/' + extension_dir)
os.getcwd()
current_folder = os.getcwd() 
list_dir = os.listdir(current_folder) 
  
# enumerate on list_dir to get the 
# content of all the folders ans store it in a dictionary
content_list = {}
for index, val in enumerate(list_dir):
    path = os.path.join(current_folder, val)
    content_list[ list_dir[index] ] = os.listdir(path)

In [109]:
merge_folder_path = current_folder
# loop through the list of folders
for sub_dir in content_list:
  
    # loop through the contents of the
    # list of folders
    for contents in content_list[sub_dir]:
  
        # make the path of the content to move 
        path_to_content = sub_dir + "/" + contents  
  
        # make the path with the current folder
        dir_to_move = os.path.join(current_folder, path_to_content )
  
        # move the file
        shutil.move(dir_to_move, merge_folder_path)

In [113]:
sourcepath = current_folder
sourcefiles = os.listdir(sourcepath)
destinationpath = path_raw # Moves all the files to the raw directory
for file in sourcefiles:
    #print(file)
    if file.endswith('.gz'):
        #print(file)
        shutil.move(os.path.join(sourcepath,file), os.path.join(destinationpath,file))

In [115]:
import glob
from sh import gunzip
os.chdir(destinationpath)
for file in glob.glob("*.gz"):
    try:
        gunzip(file)
    except:
        print('already exists')

In [118]:
shutil.rmtree(destinationpath + '/' + 'mastDownload') 